In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [2]:
# TPUs are not part of the same computer, but are connected through the cloud
# We shall connect to the cluster of TPUs in the cloud
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.74.145.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.145.42:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [3]:
# Create a TPU strategy object (like in Mirrored Strategy.ipynb)
strategy = tf.distribute.TPUStrategy(resolver)
# 8 TPU cores

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
# Note: model creation must be in strategy scope, so make a function that will be called in the scope
def create_model():
  i = Input(shape=(32, 32, 3))

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  # remember, batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required to train deep networks
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(10)(x)

  model = Model(i, x)
  return model

In [6]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

170508288/170498071 [==============================] - 2s 0us/step


In [7]:
# Use Dataset objects, which will be used when we call fit() later
# These contains optimization that speed up training, faster than using numpy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [8]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer='adam',
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['sparse_categorical_accuracy']
  )

batch_size = 256

# Split Dataset objects into batches, using the batch size
# Want to shuffle the data for training set (reshuffle_each_iteration=True by default, which is what we want)
train_dataset = train_dataset.shuffle(1000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# How to call fit() with these tf Datasets
model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset
)

Epoch 1/5
196/196 [==============================] - 24s 76ms/step - loss: 1.6179 - sparse_categorical_accuracy: 0.4806 - val_loss: 4.0499 - val_sparse_categorical_accuracy: 0.1370
Epoch 2/5
196/196 [==============================] - 9s 44ms/step - loss: 0.9551 - sparse_categorical_accuracy: 0.6620 - val_loss: 2.6538 - val_sparse_categorical_accuracy: 0.2866
Epoch 3/5
196/196 [==============================] - 8s 43ms/step - loss: 0.7327 - sparse_categorical_accuracy: 0.7420 - val_loss: 0.8910 - val_sparse_categorical_accuracy: 0.6899
Epoch 4/5
196/196 [==============================] - 9s 43ms/step - loss: 0.6018 - sparse_categorical_accuracy: 0.7873 - val_loss: 0.7055 - val_sparse_categorical_accuracy: 0.7563
Epoch 5/5
196/196 [==============================] - 10s 52ms/step - loss: 0.5067 - sparse_categorical_accuracy: 0.8220 - val_loss: 0.7024 - val_sparse_categorical_accuracy: 0.7682


In [9]:
# We're done, but in a real world scenario you may want to save the model for future use
model.save('mymodel.h5')

In [10]:
!ls

mymodel.h5  sample_data


In [11]:
# To load the model and make predictions using the TPU, put the code in scope again
with strategy.scope():
  model = tf.keras.models.load_model('mymodel.h5') 
  out = model.predict(x_test[:1])
  print(out)

[[-1.8147476  -1.8986686   0.15360616  3.976486   -1.3856671   3.557837
   2.7010221  -2.4544055  -2.1636336  -2.4343836 ]]
